## Exploratory data analysis: lobbying

In [53]:
import numpy as np
import pandas as pd
import datetime
import ast
from matplotlib import pyplot as plt
import seaborn as sns
sns.set_theme()

In [62]:
from API_Keys import get_opensecrets_key

## Sorting by lobbying issue code

To properly start looking at the lobbying data, we need to understand the lobbying codes and sort by them. Here is a dictionary called `issue_codes` relating the issue codes appearing in the lobbying filings to their meanings - it's lifted from https://lda.congress.gov/ld/help/default.htm?turl=Documents%2FAppCodes.htm.

In [40]:
issue_codes={
    'ACC': 'Accounting',
    'ADV': 'Advertising',
    'AER': 'Aerospace',
    'AGR': 'Agriculture',
    'ALC': 'Alcohol & Drug Abuse',
    'ANI': 'Animals',
    'APP': 'Apparel/Clothing Industry/Textiles',
    'ART': 'Arts/Entertainment',
    'AUT': 'Automotive Industry',
    'AVI': 'Aviation/Aircraft/Airlines',
    'BAN': 'Banking',
    'BNK': 'Bankruptcy',
    'BEV': 'Beverage Industry',
    'BUD': 'Budget/Appropriations',
    'CAW': 'Clean Air & Water (Quality)',
    'CDT': 'Commodities (Big Ticket)',
    'CHM': 'Chemicals/Chemical Industry',
    'CIV': 'Civil Rights/Civil Liberties',
    'COM': 'Communications/Broadcasting/Radio/TV',
    'CPI': 'Computer Industry',
    'CSP': 'Consumer Issues/Safety/Protection',
    'CON': 'Constitution',
    'CPT': 'Copyright/Patent/Trademark',
    'DEF': 'Defense',
    'DOC': 'District of Columbia',
    'DIS': 'Disaster Planning/Emergencies',
    'ECN': 'Economics/Economic Development',
    'EDU': 'Education',
    'ENG': 'Energy/Nuclear',
    'ENV': 'Environmental/Superfund',
    'FAM': 'Family Issues/Abortion/Adoption',
    'FIR': 'Firearms/Guns/Ammunition',
    'FIN': 'Financial Institutions/Investments/Securities',
    'FOO': 'Food Industry (Safety, Labeling, etc.)',
    'FOR': 'Foreign Relations',
    'FUE': 'Fuel/Gas/Oil',
    'GAM': 'Gaming/Gambling/Casino',
    'GOV': 'Government Issues',
    'HCR': 'Health Issues',
    'HOM': 'Homeland Security',
    'HOU': 'Housing',
    'IMM': 'Immigration',
    'IND': 'Indian/Native American Affairs',
    'INS': 'Insurance',
    'LBR': 'Labor Issues/Antitrust/Workplace',
    'INT': 'Intelligence and Surveillance',
    'LAW': 'Law Enforcement/Crime/Criminal Justice',
    'MAN': 'Manufacturing',
    'MAR': 'Marine/Maritime/Boating/Fisheries',
    'MED': 'Medical/Disease Research/Clinical Labs',
    'MIA': 'Media (Information/Publishing)',
    'MMM': 'Medicare/Medicaid',
    'MON': 'Minting/Money/Gold Standard',
    'NAT': 'Natural Resources',
    'PHA': 'Pharmacy',
    'POS': 'Postal',
    'RRR': 'Railroads',
    'RES': 'Real Estate/Land Use/Conservation',
    'REL': 'Religion',
    'RET': 'Retirement',
    'ROD': 'Roads/Highway',
    'SCI': 'Science/Technology',
    'SMB': 'Small Business',
    'SPO': 'Sports/Athletics',
    'TAR': 'Miscellaneous Tariff Bills',
    'TAX': 'Taxation/Internal Revenue Code',
    'TEC': 'Telecommunications',
    'TOB': 'Tobacco',
    'TOR': 'Torts',
    'TRD': 'Trade (Domestic & Foreign)',
    'TRA': 'Transportation',
    'TOU': 'Travel/Tourism',
    'TRU': 'Trucking/Shipping',
    'URB': 'Urban Development/Municipalities',
    'UNM': 'Unemployment',
    'UTI': 'Utilities',
    'VET': 'Veterans',
    'WAS': 'Waste (hazardous/solid/interstate/nuclear)',
    'WEL': 'Welfare'
}

Compare this to the sector breakdown Paul attached to the stock data.

In [89]:
sectors=list(pd.read_csv("trading_data/stock_sectors.csv", header=None)[0]).remove('Not Found')

In [58]:
sectors

['Advertising Agencies',
 'Aerospace & Defense',
 'Agricultural Inputs',
 'Airlines',
 'Airports & Air Services',
 'Aluminum',
 'Apparel Manufacturing',
 'Apparel Retail',
 'Asset Management',
 'Auto & Truck Dealerships',
 'Auto Manufacturers',
 'Auto Parts',
 'Banks - Diversified',
 'Banks - Regional',
 'Beverages - Brewers',
 'Beverages - Non-Alcoholic',
 'Beverages - Wineries & Distilleries',
 'Biotechnology',
 'Broadcasting',
 'Building Materials',
 'Building Products & Equipment',
 'Business Equipment & Supplies',
 'Capital Markets',
 'Chemicals',
 'Coking Coal',
 'Communication Equipment',
 'Computer Hardware',
 'Confectioners',
 'Conglomerates',
 'Consulting Services',
 'Consumer Electronics',
 'Copper',
 'Credit Services',
 'Department Stores',
 'Diagnostics & Research',
 'Discount Stores',
 'Drug Manufacturers - General',
 'Drug Manufacturers - Specialty & Generic',
 'Education & Training Services',
 'Electrical Equipment & Parts',
 'Electronic Components',
 'Electronic Gaming

This comparison is by no means one-to-one. However, there are definitely some lobbying issue codes that strongly match an industry sector. Paul did a bit of matching codes to one another - let's see.

No matter what though, we should start by breaking down the data by issue codes. First, we can keep track of filing period by returning the appropriate start and end date.

In [58]:
def filing_period_start(year,quarter):
    if quarter=='1':
        start= datetime.date(year=year, month=1, day=1)
    if quarter=='2':
        start= datetime.date(year=year, month=4, day=1)
    if quarter=='3':
        start= datetime.date(year=year, month=7, day=1)
    if quarter=='4':
        start= datetime.date(year=year, month=10, day=1)
    return start

def filing_period_end(year,quarter):
    if quarter=='1':
        d= datetime.date(year=year, month=3, day=31)
    if quarter=='2':
        d= datetime.date(year=year, month=6, day=30)
    if quarter=='3':
        d= datetime.date(year=year, month=9, day=30)
    if quarter=='4':
        d= datetime.date(year=year, month=12, day=31)
    return d


In [60]:
years=range(2013,2024) # I omit the 2024 data for now - maybe we redownload this and clean it, since it must have been updated by now to include a full reporting of Q1 information.
quarters=[str(i) for i in range(1,5)]
lobbying=pd.DataFrame()
for year in years:
    for quarter in quarters:
        df=pd.read_csv("lobbying_data/by_quarter/filings_"+str(year)+"Q"+quarter+".csv")
        df['period_start']=filing_period_start(year, quarter)
        df['period_end']=filing_period_end(year, quarter)
        lobbying=pd.concat([lobbying, df])

Let me export this as a single file, so that in the future we don't have to load in the data one quarter or issue code at a time.

In [62]:
lobbying.to_csv("lobbying_data/filings_all.csv", index=False)

In [63]:
lobbying.info()

<class 'pandas.core.frame.DataFrame'>
Index: 810672 entries, 0 to 21600
Data columns (total 18 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   registrant                810672 non-null  object 
 1   client                    810672 non-null  object 
 2   income                    810672 non-null  float64
 3   expenses                  810672 non-null  float64
 4   issue_codes               810672 non-null  object 
 5   entities_lobbied          810672 non-null  object 
 6   lobbying_description      810672 non-null  object 
 7   foreign_entities          810672 non-null  object 
 8   affiliated_organizations  810672 non-null  object 
 9   JACK_conviction           810672 non-null  bool   
 10  filing_uuid               810672 non-null  object 
 11  filing_type               810672 non-null  object 
 12  expenses_method           810672 non-null  object 
 13  dt_posted                 810672 non-null  object 

When we sort by issue code, we will ignore all filings with no attached codes.

In [18]:
no_codes=lobbying.loc[lobbying.issue_codes=="[]"].copy()

In [19]:
no_codes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 116082 entries, 2 to 21599
Data columns (total 18 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   registrant                116082 non-null  object 
 1   client                    116082 non-null  object 
 2   income                    116082 non-null  float64
 3   expenses                  116082 non-null  float64
 4   issue_codes               116082 non-null  object 
 5   entities_lobbied          116082 non-null  object 
 6   lobbying_description      116082 non-null  object 
 7   foreign_entities          116082 non-null  object 
 8   affiliated_organizations  116082 non-null  object 
 9   JACK_conviction           116082 non-null  bool   
 10  filing_uuid               116082 non-null  object 
 11  filing_type               116082 non-null  object 
 12  expenses_method           116082 non-null  object 
 13  dt_posted                 116082 non-null  object 

It's about $1/8$ of the total filings! How much is it by spending?

In [65]:
spending=np.sum(lobbying['expenses'])+np.sum(lobbying['income'])
spending_no_codes=np.sum(no_codes['expenses'])+np.sum(no_codes['income'])
print(spending_no_codes/spending)

0.010480280681845748


It's only around 1% of total spending. I don't feel too bad about ignoring this then.

In [21]:
only_codes=pd.concat([lobbying,no_codes]).drop_duplicates(keep=False)

In [30]:
only_codes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 694590 entries, 0 to 21600
Data columns (total 18 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   registrant                694590 non-null  object 
 1   client                    694590 non-null  object 
 2   income                    694590 non-null  float64
 3   expenses                  694590 non-null  float64
 4   issue_codes               694590 non-null  object 
 5   entities_lobbied          694590 non-null  object 
 6   lobbying_description      694590 non-null  object 
 7   foreign_entities          694590 non-null  object 
 8   affiliated_organizations  694590 non-null  object 
 9   JACK_conviction           694590 non-null  bool   
 10  filing_uuid               694590 non-null  object 
 11  filing_type               694590 non-null  object 
 12  expenses_method           694590 non-null  object 
 13  dt_posted                 694590 non-null  object 

In [32]:
issue_list=[ast.literal_eval(only_codes.iloc[i]['issue_codes']) for i in range(len(only_codes))]

In [33]:
only_codes['issue_codes']=issue_list

In [37]:

in_bool=[('ACC' in only_codes.iloc[i]['issue_codes']) for i in range(len(only_codes))]

In [38]:
only_codes.loc[in_bool]

,registrant,client,income,expenses,issue_codes,entities_lobbied,lobbying_description,foreign_entities,affiliated_organizations,JACK_conviction,filing_uuid,filing_type,expenses_method,dt_posted,reg_id,cl_id,period_start,period_end
430,CHS INC.,"CHS, INC",0.0,40000.00,"[ACC, ENG, BAN, AGR, TRA, TAX]","['HOUSE OF REPRESENTATIVES', 'SENATE']","['LIFO', 'RIN issue', 'End user use of derivat...",[],[],False,9095f6ef-3754-4f12-a794-af4598c36598,Q1,a,2013-04-04 14:58:54.537,286451,176515,2013-01-01,2013-03-31
443,"THE VELASQUEZ GROUP, L.L.C.",KPMG LLP,80000.0,0.00,[ACC],"['HOUSE OF REPRESENTATIVES', 'SENATE']",['Monitoring for would be legislative provisio...,[],[],False,caf4965d-4c2b-4376-b06f-d90a28cb0671,Q1,0,2013-04-04 20:34:21.480,53403,158245,2013-01-01,2013-03-31
619,UNIVERSITY OF IOWA,UNIVERSITY OF IOWA,0.0,29312.86,"[BAN, ENV, SPO, ACC, GOV, ROD, UTI, AGR, WAS, ...","['Agriculture, Dept of (USDA)', 'Centers For M...",['Implementation of the Higher Education Act a...,[],[],False,d15d1944-6d6b-4b94-9a3c-4cc502798711,Q1,a,2013-04-08 11:14:59.307,49191,156117,2013-01-01,2013-03-31
986,DALY CONSULTING GROUP,PWC US GROUP LLP (FORMERLY PRICEWATERHOUSECOOP...,45000.0,0.00,[ACC],"['HOUSE OF REPRESENTATIVES', 'SENATE', 'Securi...",['Issues related to the accounting profession.'],[],[],False,44ae954d-4525-49fe-8142-db98c67b3511,Q1,0,2013-04-09 17:27:14.800,400923683,196221,2013-01-01,2013-03-31
999,JULEANNA GLOVER,AICPA,15000.0,0.00,[ACC],['HOUSE OF REPRESENTATIVES'],['Mobile Workforce Act'],[],[],False,5712e1a9-4612-4774-a421-b235dfc8e46a,Q1,0,2013-04-09 18:23:24.117,298934,180103,2013-01-01,2013-03-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20182,MARKQUEST,MARKQUEST,0.0,1.00,"[TAX, ACC, FOO, BEV, DEF]","['Army, Dept of (Other)', 'Defense Commissary ...","['business expenses', 'how to treat business e...",[],[],False,57b46505-bf3b-497a-9e3a-cbf585c60c1a,Q4,A,2024-01-31 12:18:13,65932,162929,2023-10-01,2023-12-31
20185,MARKQUEST,KASICH COMPANY,1.0,0.00,"[TAX, SMB, ACC]","['HOUSE OF REPRESENTATIVES', 'SENATE']","['tax relief for business', 'business relief',...",[],[],False,f1a4db02-61e9-4ca7-9fe5-45ffcb6e47d9,Q4,0,2024-01-31 12:22:27,65932,204573,2023-10-01,2023-12-31
20506,THE WILLIAMS GROUP,FINANCIAL ACCOUNTING FOUNDATION,45000.0,0.00,[ACC],"['HOUSE OF REPRESENTATIVES', 'SENATE', 'Securi...",['Issues relating to the formulation and devel...,[],[],False,6f552393-10cc-4577-b5d5-96f9dfa63446,Q4,0,2024-02-05 12:36:53,401104049,54899,2023-10-01,2023-12-31
20652,THE DLM GROUP,"GENESIS HEALTH CARE, INC.",15000.0,0.00,"[MED, ALC, ACC, HCR, MMM]","['HOUSE OF REPRESENTATIVES', 'Health & Human S...","['no activity', 'Substance abuse funding', 'An...",[],[],False,e8b4951b-7b6e-454c-bc35-05a359bd2b7e,Q4,0,2024-02-05 13:13:13,313794,184703,2023-10-01,2023-12-31


Here, let me export the filings broken up in this way.

In [43]:
for issue in issue_codes.keys():
    issue_bool=[(issue in only_codes.iloc[i]['issue_codes']) for i in range(len(only_codes))]
    lobbying_issue=only_codes.loc[issue_bool].copy()
    lobbying_issue.to_csv("lobbying_data/by_issue_code/filings_"+issue+".csv", index=False)

In [92]:
summary=pd.DataFrame()
for issue in issue_codes.keys():
    issue_bool=[(issue in only_codes.iloc[i]['issue_codes']) for i in range(len(only_codes))]
    lobbying_issue=only_codes.loc[issue_bool].copy()
    lobbying_issue['spending']=lobbying_issue['expenses']+lobbying_issue['income']
    spending_per_quarter=lobbying_issue.groupby(['period_start']).spending.sum()
    summary[issue]=spending_per_quarter

And we export it.

In [99]:
summary.to_csv("lobbying_data/lobbying_totals.csv")

In [ ]:
for issue in issue_codes.keys():
    plt.figure(figsize=(5,5))
    plt.title("Lobbying for "+issue_codes[issue])
    plt.plot(summary.index, summary[issue])
    plt.xlabel("Quarter (start date)")
    plt.ylabel(issue+" Spending (USD)")


In [131]:
sec2issue=pd.read_csv("trading_data/sector2lobbyingcode.csv",header=None)
sec2issue.columns=['stock_sector', 'issue_code_1', 'issue_code_2', 'issue_code_3']
sec2issue=sec2issue.fillna(0)

In [152]:
sec2issue=pd.read_csv("trading_data/sector2lobbyingcode.csv", header=None).fillna(0)
sec2issue.columns=['sector', 'code1', 'code2', 'code3']

In [153]:
sec2issue

,sector,code1,code2,code3
0,Advertising Agencies,ADV,0,0
1,Aerospace & Defense,AER,DEF,0
2,Agricultural Inputs,AGR,0,0
3,Airlines,AER,TRA,TOU
4,Airports & Air Services,AER,TRA,TOU
...,...,...,...,...
141,Utilities - Regulated Electric,UTI,ENG,0
142,Utilities - Regulated Gas,UTI,ENG,FUE
143,Utilities - Regulated Water,UTI,0,0
144,Utilities - Renewable,UTI,0,0


In [161]:
one_code=sec2issue.loc[(sec2issue.code2==0) &  (sec2issue.code3==0) & (sec2issue.code1 !=0)]

In [178]:
codes=list(set(one_code['code1'].values))
codes.sort()
codes

['ADV',
 'AGR',
 'APP',
 'ART',
 'AUT',
 'BEV',
 'CHM',
 'CPI',
 'EDU',
 'ENG',
 'FIN',
 'FOO',
 'GAM',
 'HCR',
 'INS',
 'LBR',
 'MAN',
 'NAT',
 'PHA',
 'RES',
 'RRR',
 'SCI',
 'TEC',
 'TOB',
 'TRU',
 'UTI',
 'WAS']

In [179]:
for code in codes:
    print( "'"+code+"'" + " : ", list(one_code.loc[one_code.code1==code]['sector']))

'ADV' :  ['Advertising Agencies']
'AGR' :  ['Agricultural Inputs', 'Farm & Heavy Construction Machinery', 'Farm Products']
'APP' :  ['Apparel Manufacturing', 'Apparel Retail', 'Department Stores', 'Discount Stores', 'Footwear & Accessories', 'Luxury Goods']
'ART' :  ['Electronic Gaming & Multimedia', 'Entertainment', 'Publishing']
'AUT' :  ['Auto & Truck Dealerships', 'Auto Manufacturers', 'Auto Parts', 'Recreational Vehicles']
'BEV' :  ['Beverages - Brewers', 'Beverages - Non-Alcoholic', 'Beverages - Wineries & Distilleries']
'CHM' :  ['Chemicals', 'Specialty Chemicals']
'CPI' :  ['Computer Hardware', 'Consumer Electronics', 'Electronics & Computer Distribution', 'Information Technology Services', 'Software - Application', 'Software - Infrastructure']
'EDU' :  ['Education & Training Services']
'ENG' :  ['Solar']
'FIN' :  ['Asset Management', 'Capital Markets', 'Financial Conglomerates', 'Financial Data & Stock Exchanges']
'FOO' :  ['Confectioners', 'Food Distribution', 'Packaged Foods

In [136]:
sec_one_code=sec2issue.loc[(sec2issue.issue_code_2==0) & (sec2issue.issue_code_3==0)].copy()


OK, here is a first list of codes to look at.

In [180]:
cognates={'ADV' :  ['Advertising Agencies'],'BEV' :  ['Beverages - Brewers', 'Beverages - Non-Alcoholic', 'Beverages - Wineries & Distilleries'], 'CHM' :  ['Chemicals', 'Specialty Chemicals'], 'FOO' :  ['Confectioners', 'Food Distribution', 'Packaged Foods'],'INS' :  ['Insurance - Diversified', 'Insurance - Life', 'Insurance - Property & Casualty', 'Insurance - Reinsurance', 'Insurance - Specialty', 'Insurance Brokers'],'TOB' :  ['Tobacco']}

In [181]:
cognates.keys()

dict_keys(['ADV', 'BEV', 'CHM', 'FOO', 'INS', 'TOB'])